In [2]:
import pandas as pd 
import numpy as np
from pathlib import Path

In [3]:
DATA_ROOT = Path("../data") / "jigsaw"

In [9]:
train = pd.read_csv(DATA_ROOT / "train.csv")
test = pd.read_csv(DATA_ROOT / "test_proced.csv")

In [6]:
train["toxic"].value_counts()

0    144277
1     15294
Name: toxic, dtype: int64

In [10]:
test["toxic"].value_counts()

0    57888
1     6090
Name: toxic, dtype: int64

Toxic ratio

In [7]:
train["toxic"].sum() / train.shape[0]

0.09584448302009764

In [11]:
test["toxic"].sum() / test.shape[0]

0.09518897120885304

### Train simple classifier

A simple classifier can tell us a lot about how difficult a certain task might be

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [13]:
vec = TfidfVectorizer(max_features=120000)
X_train = vec.fit_transform(train["comment_text"])

In [15]:
X_train.shape

(159571, 120000)

In [14]:
X_test = vec.transform(test["comment_text"])

In [16]:
lr = LogisticRegression()

In [18]:
lr.fit(X_train, train["toxic"])

/Users/keitakurita/.local/share/virtualenvs/NNforNLP-axlxMEpb/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

Check results

In [19]:
from sklearn.metrics import f1_score, roc_auc_score

In [23]:
roc_auc_score(train["toxic"], lr.predict_proba(X_train)[:, 1])

0.9843896565973511

In [24]:
roc_auc_score(test["toxic"], lr.predict_proba(X_test)[:, 1])

0.9592978678719157

In [25]:
f1_score(train["toxic"], lr.predict_proba(X_train)[:, 1] > 0.5)

0.7688682889146389

In [27]:
f1_score(test["toxic"], lr.predict_proba(X_test)[:, 1] > 0.5)

0.6804477489991012

Check coefficients

{'explanation': 29034,
 'why': 111446,
 'the': 97178,
 'edits': 26361,
 'made': 48417,
 'under': 102918,
 'my': 53434,
 'username': 105252,
 'hardcore': 36163,
 'metallica': 50892,
 'fan': 29676,
 'were': 110654,
 'reverted': 74859,
 'they': 97509,
 'weren': 110663,
 'vandalisms': 106124,
 'just': 43283,
 'closure': 17373,
 'on': 56945,
 'some': 87561,
 'gas': 32989,
 'after': 4556,
 'voted': 108621,
 'at': 8316,
 'new': 54660,
 'york': 116015,
 'dolls': 24837,
 'fac': 29357,
 'and': 6157,
 'please': 63613,
 'don': 24900,
 'remove': 73768,
 'template': 96395,
 'from': 32140,
 'talk': 95523,
 'page': 58297,
 'since': 85080,
 'retired': 74612,
 'now': 55788,
 '89': 2809,
 '205': 1215,
 '38': 1823,
 '27': 1491,
 'aww': 8995,
 'he': 36590,
 'matches': 49688,
 'this': 97604,
 'background': 9248,
 'colour': 17920,
 'seemingly': 81335,
 'stuck': 91806,
 'with': 113042,
 'thanks': 97068,
 '21': 1297,
 '51': 2166,
 'january': 42230,
 '11': 289,
 '2016': 1195,
 'utc': 105440,
 'hey': 37207,
 'ma

In [35]:
np.array(["a"])

array(['a'], dtype='<U1')

In [43]:
index_to_word = ['' for _ in vec.vocabulary_]
for w, i in vec.vocabulary_.items():
    index_to_word[i] = w
index_to_word = np.array(index_to_word)

Most benevolent words

In [51]:
index_to_word[lr.coef_.argsort()][0, :20]

array(['thank', 'thanks', 'please', 'redirect', 'talk', 'if', 'help',
       'may', 'for', 'at', 'sorry', 'welcome', 'agree', 'best', 'could',
       'interested', 'there', 'appreciate', 'cheers', 'consensus'],
      dtype='<U4931')

Most malicious words

In [52]:
index_to_word[lr.coef_.argsort()][0, -20:]

array(['bastard', 'pathetic', 'moron', 'idiots', 'penis', 'hell',
       'faggot', 'sucks', 'dick', 'bitch', 'crap', 'asshole', 'suck',
       'bullshit', 'ass', 'idiot', 'stupid', 'shit', 'fucking', 'fuck'],
      dtype='<U4931')